In [ ]:
from scipy.cluster.vq import vq
import numpy as np
import glob
from skimage.feature import SIFT
import cv2
from PIL.Image import Image
from matplotlib import pyplot as plt

bishop_path = glob.glob("bishop_resized/*.jpg", recursive=True)
knight_path = glob.glob("knight-resize/*.jpg", recursive=True)
pawn_path = glob.glob("pawn_resized/*.jpg", recursive=True)
queen_path = glob.glob("Queen-resized/*.jpg", recursive=True)
rook_path = glob.glob("Rook-resize/*.jpg", recursive=True)

image_paths = bishop_path + knight_path + pawn_path + queen_path + rook_path

images = [cv2.imread(image_path, cv2.IMREAD_GRAYSCALE) for image_path in image_paths]

sift = SIFT()
descriptors = []

images_valid = []
descriptors = []

for img in images:
    try:
        sift.detect_and_extract(img)
        if sift.descriptors is not None and len(sift.descriptors) > 0:
            descriptors.append(sift.descriptors)
            images_valid.append(img)  # μόνο τις χρήσιμες
    except RuntimeError:
        print("SIFT failed on one image. Skipping.")

all_descriptors = np.vstack([desc for desc in descriptors if desc is not None]).astype(float)

# -------------------- 4. Εκπαίδευση KMeans (λεξικό) --------------------
k = 100  # Αριθμός visual words
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
attempts = 10
flags = cv2.KMEANS_RANDOM_CENTERS

all_descriptors = all_descriptors.astype(np.float32)

compactness, labels, centers = cv2.kmeans(all_descriptors, k, None, criteria, attempts, flags)
codebook = centers

# -------------------- 5. Αντιστοίχιση descriptors σε visual words --------------------
visual_words = []
for desc in descriptors:
    if desc is not None:
        words, _ = vq(desc, codebook)
    else:
        words = np.array([])
    visual_words.append(words)

# -------------------- 6. Δημιουργία BoVW vector (ιστόγραμμα) --------------------
bovw_vectors = []
for words in visual_words:
    hist = np.zeros(k)
    for w in words:
        hist[w] += 1
    bovw_vectors.append(hist)

bovw_vectors = np.stack(bovw_vectors)

# -------------------- 7. (Προαιρετικά) TF-IDF μετασχηματισμός --------------------
N = len(bovw_vectors)
df = np.sum(bovw_vectors > 0, axis=0)
idf = np.log(N / (df + 1e-6))  # μικρή τιμή για αποφυγή log(0)
tf_idf = bovw_vectors * idf

# -------------------- 8. Προβολή αποτελεσμάτων --------------------


SIFT failed on one image. Skipping.
SIFT failed on one image. Skipping.
SIFT failed on one image. Skipping.
